In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample, shuffle
from scipy.sparse import vstack

import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv('dataset/amazon-fine-food-reviews/Reviews.csv')
print(f"Dataset shape: {df.shape}")
df.head()
